In [50]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import KernelPCA, PCA

from factor_analyzer import FactorAnalyzer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


from sklearn import preprocessing  
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [51]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [65]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [53]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [66]:
df = pd.read_excel('airfoil_self_noise.xlsx')
df.head()

df.shape   
df.info() 
X_input = df.dropna()
scaler = MinMaxScaler()

X_input = scaler.fit_transform(X_input)
X_input = pd.DataFrame(X_input)
X_input.columns = ["Frequency","Angle of attack","Chord length","Free-stream velocity","Suction side displacement thickness","label","Scaled sound pressure level"] #添加自定义的columns的名字


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      1503 non-null   int64  
 1   x2      1503 non-null   float64
 2   x3      1503 non-null   float64
 3   x4      1503 non-null   float64
 4   x5      1503 non-null   float64
 5   label   1503 non-null   int64  
 6   y       1503 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 82.3 KB


In [67]:
X = X_input.iloc[:,0:6]
y = X_input.iloc[:,6]

In [68]:
X

,Frequency,Angle of attack,Chord length,Free-stream velocity,Suction side displacement thickness,label
0,0.030303,0.000000,1.000000,1.000000,0.039005,0.0
1,0.040404,0.000000,1.000000,1.000000,0.039005,0.0
2,0.053030,0.000000,1.000000,1.000000,0.039005,0.0
3,0.070707,0.000000,1.000000,1.000000,0.039005,0.0
4,0.090909,0.000000,1.000000,1.000000,0.039005,0.0
...,...,...,...,...,...,...
1498,0.116162,0.702703,0.272727,0.199495,0.904111,1.0
1499,0.148990,0.702703,0.272727,0.199495,0.904111,1.0
1500,0.191919,0.702703,0.272727,0.199495,0.904111,1.0
1501,0.242424,0.702703,0.272727,0.199495,0.904111,1.0


In [69]:
y

0       0.606829
1       0.580238
2       0.600181
3       0.643790
4       0.640333
          ...   
1498    0.183051
1499    0.156194
1500    0.085729
1501    0.075624
1502    0.021911
Name: Scaled sound pressure level, Length: 1503, dtype: float64

In [70]:
list1=[[0,330],[331,645],[646,934],[935,1225],[1226,1503]]
a=[]
for i in list1:
    a.append(i[0])
print(a)

[0, 331, 646, 935, 1226]


In [73]:
ols_score=[]
ridge_Training_score=[]
ridge_Testing_score=[]
lasso_Training_score=[]
lasso_Testing_score=[]
DTRscore=[]
SVRscore=[]
RFscore=[]
adbscore=[]
gbtrscore=[]
MLPscore=[]
for i in list1:
    X_test=X.iloc[i[0]:i[1]]
    y_test=y.iloc[i[0]:i[1]]
    X_train= X.drop(X.index[i[0]:i[1]])
    y_train= y.drop(y.index[i[0]:i[1]])
    x = sm.add_constant(X_train) 
    model = sm.OLS(y_train, x).fit() 
    ols_score.append(mean_squared_error(y_test,model.predict(sm.add_constant(X_test))))
    
    ridge=Ridge(alpha=0.4)
    ridge.fit(X_train,y_train)
    ridge_Training_score.append(mean_squared_error(y_train,ridge.predict(X_train)))
    ridge_Testing_score.append(mean_squared_error(y_test,ridge.predict(X_test)))

    lasso_cv = LassoCV(cv=5)
    lasso_cv.fit(X_train, y_train)
    train_score=lasso_cv.score(X_train, y_train)
    test_score = lasso_cv.score(X_test, y_test)
    lasso_Training_score.append(mean_squared_error(y_train,lasso_cv.predict(X_train)))
    lasso_Testing_score.append(mean_squared_error(y_test,lasso_cv.predict(X_test)))
    
    DTR = DecisionTreeRegressor(max_depth=30)
    DTR.fit(X_train,y_train)
    y_pre_DTR = DTR.predict(X_test)
   # DTR_score=r2_score(y_test,y_pre_DTR)
    DTRscore.append((mean_squared_error(y_test,y_pre_DTR)))
    
    SUPPOT = SVR()
    SUPPOT.fit(X_train,y_train)
    y_pre_SVR = SUPPOT.predict(X_test)
    SVRscore.append((mean_squared_error(y_test,y_pre_SVR)))

    
    rf = RandomForestRegressor(n_estimators=30,max_depth=20)
    rf.fit(X_train,y_train)
    y_pre_rf = rf.predict(X_test)
    #rf_score=r2_score(y_test,y_pre_rf)
    RFscore.append((mean_squared_error(y_test,y_pre_rf)))
    
    adb = AdaBoostRegressor()
    adb.fit(X_train,y_train)
    y_pre_adb = adb.predict(X_test)
    #adb_score=r2_score(y_test,y_pre_adb)
    adbscore.append((mean_squared_error(y_test,y_pre_adb)))
    
    gbtr = GradientBoostingRegressor()
    gbtr.fit(X_train,y_train)
    y_pre_gbtr = gbtr.predict(X_test)
    #gbtr_score=r2_score(y_test,y_pre_gbtr)
    gbtrscore.append((mean_squared_error(y_test,y_pre_gbtr)))
    
    test_score = []
    layers = list(range(5,300,5)) 
    for i in layers:
        MLP = MLPRegressor(activation = 'relu', 
                           solver = 'sgd', 
                           hidden_layer_sizes = (i,), 
                           alpha=1e-2, 
                           max_iter = 400,
                           learning_rate_init = 0.1)
        MLP.fit(X_train,y_train)
        y_pre=MLP.predict(X_test)
        test_score.append(mean_squared_error(y_test,y_pre))
    MLPscore.append(min(test_score))

C:\Users\jingu\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\jingu\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\jingu\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\jingu\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [74]:
print(np.mean(ols_score))
print(np.mean(ridge_Training_score))
print(np.mean(ridge_Testing_score))
print(np.mean(lasso_Training_score))
print(np.mean(lasso_Testing_score))

0.021143616025547166
0.01590857617589644
0.020501444086477853
0.015987126830970755
0.01977122185016922


In [75]:
print(ols_score)
print((ridge_Training_score))
print((ridge_Testing_score))
print((lasso_Training_score))
print((lasso_Testing_score))

[0.006605979498060085, 0.01673021408204692, 0.013658162718292914, 0.043663796826784514, 0.025059927002551403]
[0.018952885929570812, 0.016278928695527474, 0.016816055261458383, 0.011430479642261559, 0.016064531350663974]
[0.0066660596249391866, 0.016868347696009853, 0.013696041833595733, 0.04297812807191989, 0.022298643205924618]
[0.0190600144238177, 0.01630655409517161, 0.016813220487275876, 0.011460388598247658, 0.016295456550340925]
[0.006970561242302446, 0.017092946428120162, 0.013663028362448336, 0.043011528616300594, 0.01811804460167455]


In [62]:
print((DTRscore))
print((SVRscore))
print((RFscore))
print((adbscore))
print((gbtrscore))
print((MLPscore))

[0.011855520134964739, 0.01813214777660629, 0.02286209008629277, 0.024599055431642917, 0.02167430156093483]
[0.006596602472094272, 0.009104803625471265, 0.01121268787220606, 0.0292612428033585, 0.028101195581525514]
[0.0047519773536334456, 0.0115198179859235, 0.011218397653291792, 0.02253286790053006, 0.017562789042809886]
[0.00783727942036717, 0.01278966435461803, 0.019960812512849542, 0.02950569632517757, 0.030533118492933303]
[0.004456619489800928, 0.009261001609651034, 0.013847953782702389, 0.022568022160093727, 0.020180583767607462]
[0.004796476655411321, 0.01332983428836662, 0.011412611967937055, 0.03628651595625619, 0.016154824343625446]


In [63]:
print(np.mean(DTRscore))
print(np.mean(SVRscore))
print(np.mean(RFscore))
print(np.mean(adbscore))
print(np.mean(gbtrscore))
print(np.mean(MLPscore))


0.01982462299808831
0.01685530647093112
0.013517169987237737
0.02012531422118912
0.014062836161971109
0.016396052642319324
